In [1]:
import numpy as np   #Used for working with arrays
import pandas as pd   # Used for data analysis

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies=movies.merge(credits,on='title')

In [4]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [6]:
movies.dropna(inplace=True)

In [7]:
movies.duplicated().sum()

0

In [8]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [9]:
import ast #This module helps python application to process trees of the python abstract syntax grammar


In [10]:
movies['genres']=movies['genres'].apply(convert)

In [11]:
movies['keywords']=movies['keywords'].apply(convert)

In [12]:
def convert3(obj):
    L=[]
    counter=0;
    for i in ast.literal_eval(obj):
        if(counter!=3):
            counter+=1
            L.append(i['name'])
        else:
            break
    return L

In [13]:
movies['cast']=movies['cast'].apply(convert3)

In [14]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if(i['job']=='Director'):
            L.append(i['name'])
            break;
            
        
    return L

In [15]:
movies['crew']=movies['crew'].apply(fetch_director)

In [16]:
movies['overview']=movies['overview'].apply(lambda x: x.split())

In [17]:
movies['genres']=movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [18]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [19]:
new_df=movies[['movie_id','title','tags']]

In [20]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_16328\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [21]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_16328\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [22]:
import nltk

In [23]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [24]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [25]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\HP\AppData\Local\Temp\ipykernel_16328\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [27]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
similarity=cosine_similarity(vectors)

In [30]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])
    
    
    for i in movies_list[0:6]:
        print(new_df.iloc[i[0]].title)

In [31]:
import pickle

In [34]:
pickle.dump(new_df,open('movies.pkl','wb'))
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [35]:
new_df

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...
